In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import pandas as pd

hdb_link = "https://services2.hdb.gov.sg/webapp/BB33RTIS/BB33SComparator"
hdb_title = "Resale Flat Prices"
user_profile_title = "userProfileB"
flat_type_title = "FLAT_TYPE"
flat_type_val = Keys.NUMPAD4
town_title = "NME_NEWTOWN"
# town_val = None
town_val = "a"
street_title = "NME_STREET"
street_val = None
# street_val = "Ang Mo Kio Ave 3"
date_range_title = "dteRange"
submit_btn_id = "btnSubmit"

params_hdb = {"hdb_link":hdb_link,
          "hdb_title":hdb_title,
          "user_profile_title":user_profile_title,
          "flat_type_title":flat_type_title,
          "flat_type_val":flat_type_val,
          "town_title":town_title,
          "town_val":town_val,
          "street_title":street_title,
          "street_val":street_val,
          "date_range_title":date_range_title,
          "submit_btn_id":submit_btn_id}

In [2]:
# start the driver, returns a webdriver chrome object
def start_driver() -> webdriver.Chrome:
    driver = webdriver.Chrome()
    driver.get(hdb_link)
    assert hdb_title in driver.title
    return driver

In [3]:
# runs the query in the website with HDB town 
def run_query_hdb(driver: webdriver.Chrome, params:dict):
    print("running hdb")
    hdb_town = driver.find_element(By.NAME, params["town_title"])
    hdb_town.send_keys(params["town_val"])


# runs the query in the website with Street Name
def run_query_street(driver: webdriver.Chrome, params:dict):
    print("running street")
    street_field = driver.find_element(By.NAME , params["street_title"])
    street_field.send_keys(params["street_val"])

    
# runs the base query that is similar to both hdb or street query
def run_base_query(driver: webdriver.Chrome, params:dict):
    radio = driver.find_element(By.ID, params["user_profile_title"])
    radio.click()
    flat_type = driver.find_element(By.NAME, params["flat_type_title"])
    flat_type.send_keys(params["flat_type_val"])

    date_range_field = driver.find_element(By.ID , params["date_range_title"])
    date_range_field.send_keys("last 12")

# master query function
# returns all records in chronological order
def run_query(driver: webdriver.Chrome, params: dict):
    if params["street_val"]:
        run_query_street(driver, params)
    else:
        run_query_hdb(driver, params)
    run_base_query(driver, params)
    submit_btn = driver.find_element(By.ID, params["submit_btn_id"])
    submit_btn.click()

In [4]:
# Runs the query twice which for some reason will then only be able to generate the results
def run_query_success(driver: webdriver.Chrome, params:dict):
    run_query(driver, params)
    run_query(driver, params)

In [5]:
driver = start_driver()
run_query_success(driver, params_hdb)


running hdb
running hdb


In [6]:
# for HDB Town
headers_hdb = [
    "Block",
    "Street Name",
    "Storey",
    "Floor Area (sqm)" , 
    "Flat Model",
    "Lease Commence Date",
    "Remaining Lease",
    "Resale Price",
    "Resale Registration Date"
    ]

In [7]:
addresses = driver.find_elements(By.XPATH, '//tr[@height="30"]')
# we only need half of them as the rest of them are repeated but not shown (ie the values are hidden)
addresses = addresses[:int(len(addresses)/2)]

In [8]:
addresses_tr = addresses[1].find_elements(By.XPATH,".//child::td")

In [9]:
len(addresses)

354

In [10]:
len(addresses_tr)

8

In [11]:
for i, address in enumerate(addresses_tr):
    print("index is :" + str(i) + " value  " + address.text)

index is :0 value  411
index is :1 value   Ang Mo Kio Ave 10
index is :2 value  07 to 09
index is :3 value  92.00
New Generation
index is :4 value  1979
index is :5 value  54 years
11 months
index is :6 value  $480,000.00
index is :7 value  Nov 2023


In [12]:
data = []

In [13]:
counter=0
for address in addresses:
    if counter >3:
        break
    tr = address.find_elements(By.XPATH,".//child::td")
    
    block = tr[0].text
    print(tr[0])
    street_name = tr[1].text
    storey = tr[2].text
    floor_area,flat_model = tr[3].text.split("\n")
    lease_commence = tr[4].text
    remaining_lease = tr[5].text.split("\n")[0]
    print(remaining_lease)
    resale_price = tr[6].text
    resale_date = tr[7].text
    data.append([block, street_name, storey, floor_area, flat_model, lease_commence, remaining_lease, resale_price, resale_date])
    counter +=1


<selenium.webdriver.remote.webelement.WebElement (session="1c63dfb8f072c9d17d026aab2d4c2768", element="2A3AD8A79AE4B51D538F0257EEF0A06F_element_1251")>
87 years


<selenium.webdriver.remote.webelement.WebElement (session="1c63dfb8f072c9d17d026aab2d4c2768", element="2A3AD8A79AE4B51D538F0257EEF0A06F_element_1241")>
54 years
<selenium.webdriver.remote.webelement.WebElement (session="1c63dfb8f072c9d17d026aab2d4c2768", element="2A3AD8A79AE4B51D538F0257EEF0A06F_element_1258")>
76 years
<selenium.webdriver.remote.webelement.WebElement (session="1c63dfb8f072c9d17d026aab2d4c2768", element="2A3AD8A79AE4B51D538F0257EEF0A06F_element_1266")>
93 years


In [14]:
data

[['310A',
  ' Ang Mo Kio Ave 1',
  '04 to 06',
  '94.00',
  'Model A',
  '2012',
  '87 years',
  '$770,000.00',
  'Nov 2023'],
 ['411',
  ' Ang Mo Kio Ave 10',
  '07 to 09',
  '92.00',
  'New Generation',
  '1979',
  '54 years',
  '$480,000.00',
  'Nov 2023'],
 ['354',
  ' Ang Mo Kio St 32',
  '13 to 15',
  '95.00',
  'Model A',
  '2001',
  '76 years',
  '$770,000.00',
  'Nov 2023'],
 ['455A',
  ' Ang Mo Kio St 44',
  '04 to 06',
  '93.00',
  'Model A',
  '2018',
  '93 years',
  '$765,000.00',
  'Nov 2023']]

In [15]:
pd.DataFrame(data, columns=headers_hdb)

,Block,Street Name,Storey,Floor Area (sqm),Flat Model,Lease Commence Date,Remaining Lease,Resale Price,Resale Registration Date
0,310A,Ang Mo Kio Ave 1,04 to 06,94.00,Model A,2012,87 years,"$770,000.00",Nov 2023
1,411,Ang Mo Kio Ave 10,07 to 09,92.00,New Generation,1979,54 years,"$480,000.00",Nov 2023
2,354,Ang Mo Kio St 32,13 to 15,95.00,Model A,2001,76 years,"$770,000.00",Nov 2023
3,455A,Ang Mo Kio St 44,04 to 06,93.00,Model A,2018,93 years,"$765,000.00",Nov 2023


In [16]:
# writes the hdb data from the HDB category coupled with the headers and returns a pd.dataframe
def write_to_dataframe_hdb(driver:webdriver.Chrome, headers:list)->pd.DataFrame:
    addresses = driver.find_elements(By.XPATH, '//tr[@height="30"]')
    # we only need half of them as the rest of them are repeated but not shown (ie the values are hidden)
    addresses = addresses[:int(len(addresses)/2)]
    data = []
    for address in addresses:
        tr = address.find_elements(By.XPATH,".//child::td")        
        block = tr[0].text
        street_name = tr[1].text
        storey = tr[2].text
        floor_area,flat_model = tr[3].text.split("\n")
        lease_commence = tr[4].text
        remaining_lease = tr[5].text.split("\n")[0]
        resale_price = tr[6].text
        resale_date = tr[7].text
        data.append([block, street_name, storey, floor_area, flat_model, lease_commence, remaining_lease, resale_price, resale_date])
    
    return pd.DataFrame(data, columns=headers)


In [17]:
df = write_to_dataframe_hdb(driver, headers_hdb)

In [18]:
df.to_csv("hdb.csv")

In [19]:
sasd

NameError: name 'sasd' is not defined

In [ ]:
hdb_link = "https://services2.hdb.gov.sg/webapp/BB33RTIS/BB33SComparator"
hdb_title = "Resale Flat Prices"
user_profile_title = "userProfileB"
flat_type_title = "FLAT_TYPE"
flat_type_val = Keys.NUMPAD4
town_title = "NME_NEWTOWN"
town_val = None
# town_val = "a"
street_title = "NME_STREET"
# street_val = None
street_val = "Ang Mo Kio Ave 3"
date_range_title = "dteRange"
submit_btn_id = "btnSubmit"

params_street = {"hdb_link":hdb_link,
          "hdb_title":hdb_title,
          "user_profile_title":user_profile_title,
          "flat_type_title":flat_type_title,
          "flat_type_val":flat_type_val,
          "town_title":town_title,
          "town_val":town_val,
          "street_title":street_title,
          "street_val":street_val,
          "date_range_title":date_range_title,
          "submit_btn_id":submit_btn_id}

In [ ]:
driver_street = start_driver()
run_query_success(driver_street, params_street)

running street
running street


In [ ]:
# for Street
headers_street = [
    "Block",
    "HDB Town",
    "Storey",
    "Floor Area (sqm)" , 
    "Flat Model",
    "Lease Commence Date",
    "Remaining Lease",
    "Resale Price",
    "Resale Registration Date"
    ]

In [ ]:
addresses = driver_street.find_elements(By.XPATH, '//div[@class="grid-container hide-for-small-only"]//tr')

In [ ]:
len(addresses)

57

In [ ]:
# should not take the first element because it consists of the header that does not have td but is th instead
addresses[0].find_elements(By.TAG_NAME,"td")

[]

In [ ]:
addresses[1].find_elements(By.TAG_NAME,"td")[1].text

'324'

In [ ]:
addresses[56].find_elements(By.TAG_NAME,"td")[1].text

'572'

In [ ]:
addresses_tr = addresses[1].find_elements(By.TAG_NAME,"td")

In [ ]:
len(addresses_tr)

8

In [ ]:
addresses_tr[7].text

'Oct 2023'

In [ ]:
for i, address in enumerate(addresses_tr):
    print("index is :" + str(i) + " value  " + address.text)

index is :0 value  Ang Mo Kio
index is :1 value  324
index is :2 value  10 to 12
index is :3 value  98.00
New Generation
index is :4 value  1978
index is :5 value  54 years
index is :6 value  $553,000.00
index is :7 value  Oct 2023


In [ ]:
# we only need the second to the last
addresses = addresses[1:len(addresses)]

In [ ]:
data = []

In [ ]:
for index ,address in enumerate(addresses):
    if index >3:
        break
    tr = address.find_elements(By.TAG_NAME,"td")
    # for tr_data in tr:
    #     print(tr_data.text)
    
    block = tr[1].text
    print(tr[0])
    hdb_town = tr[0].text
    storey = tr[2].text
    floor_area,flat_model = tr[3].text.split("\n")
    lease_commence = tr[4].text
    remaining_lease = tr[5].text.split("\n")[0]
    print(remaining_lease)
    resale_price = tr[6].text
    resale_date = tr[7].text
    data.append([block, hdb_town, storey, floor_area, flat_model, lease_commence, remaining_lease, resale_price, resale_date])


<selenium.webdriver.remote.webelement.WebElement (session="ea687362ec49e4bc80d824f52a95a16e", element="608FFA28C571093C1385D76D65243293_element_290")>
54 years
<selenium.webdriver.remote.webelement.WebElement (session="ea687362ec49e4bc80d824f52a95a16e", element="608FFA28C571093C1385D76D65243293_element_306")>
54 years
<selenium.webdriver.remote.webelement.WebElement (session="ea687362ec49e4bc80d824f52a95a16e", element="608FFA28C571093C1385D76D65243293_element_314")>


52 years
<selenium.webdriver.remote.webelement.WebElement (session="ea687362ec49e4bc80d824f52a95a16e", element="608FFA28C571093C1385D76D65243293_element_322")>
53 years


In [ ]:
data

[['324',
  'Ang Mo Kio',
  '10 to 12',
  '98.00',
  'New Generation',
  '1978',
  '54 years',
  '$553,000.00',
  'Oct 2023'],
 ['122',
  'Ang Mo Kio',
  '07 to 09',
  '92.00',
  'New Generation',
  '1978',
  '54 years',
  '$492,000.00',
  'Sep 2023'],
 ['210',
  'Ang Mo Kio',
  '01 to 03',
  '91.00',
  'New Generation',
  '1977',
  '52 years',
  '$470,000.00',
  'Sep 2023'],
 ['348',
  'Ang Mo Kio',
  '07 to 09',
  '89.00',
  'New Generation',
  '1978',
  '53 years',
  '$500,000.00',
  'Sep 2023']]

In [ ]:
headers_street

['Block',
 'HDB Town',
 'Storey',
 'Floor Area (sqm)',
 'Flat Model',
 'Lease Commence Date',
 'Remaining Lease',
 'Resale Price',
 'Resale Registration Date']

In [ ]:
pd.DataFrame(data, columns=headers_street)

,Block,HDB Town,Storey,Floor Area (sqm),Flat Model,Lease Commence Date,Remaining Lease,Resale Price,Resale Registration Date
0,324,Ang Mo Kio,10 to 12,98.00,New Generation,1978,54 years,"$553,000.00",Oct 2023
1,122,Ang Mo Kio,07 to 09,92.00,New Generation,1978,54 years,"$492,000.00",Sep 2023
2,210,Ang Mo Kio,01 to 03,91.00,New Generation,1977,52 years,"$470,000.00",Sep 2023
3,348,Ang Mo Kio,07 to 09,89.00,New Generation,1978,53 years,"$500,000.00",Sep 2023


In [ ]:
# writes the hdb data from the Street category coupled with the headers and returns a pd.dataframe
def write_to_dataframe_street(driver:webdriver.Chrome, headers:list)->pd.DataFrame:
    addresses = driver.find_elements(By.XPATH, '//div[@class="grid-container hide-for-small-only"]//tr')
    # we only need the second to the last
    addresses = addresses[1:len(addresses)]
    data = []
    for address in addresses:
        tr = address.find_elements(By.TAG_NAME,"td")      
        block = tr[1].text
        hdb_town = tr[0].text
        storey = tr[2].text
        floor_area,flat_model = tr[3].text.split("\n")
        lease_commence = tr[4].text
        remaining_lease = tr[5].text.split("\n")[0]
        print(remaining_lease)
        resale_price = tr[6].text
        resale_date = tr[7].text
        data.append([block, hdb_town, storey, floor_area, flat_model, lease_commence, remaining_lease, resale_price, resale_date])
    
    return pd.DataFrame(data, columns=headers)


In [ ]:
# master write dataframe function
def write_to_dataframe(driver:webdriver.Chrome,params:dict,headers:list) -> pd.DataFrame:
    print(params["street_val"])
    if params["street_val"]:
        print('params["street_val"] is true')
        return write_to_dataframe_street(driver, headers)
    else:
        print('params["street_val"] is false')
        return write_to_dataframe_hdb(driver, headers)

In [ ]:
write_to_dataframe_street(driver_street, headers_street)

54 years
54 years
52 years
53 years
54 years
68 years
52 years
52 years
54 years
68 years
52 years
53 years
53 years
53 years
54 years
54 years
54 years
53 years
54 years
52 years
53 years
53 years
53 years
53 years
55 years
54 years
54 years
54 years
54 years
53 years
53 years
55 years
54 years
53 years
54 years
54 years
55 years
54 years
54 years
54 years
54 years
54 years
54 years
54 years
53 years
54 years
54 years
54 years
55 years
53 years
54 years
55 years
54 years
54 years
53 years
55 years


,Block,HDB Town,Storey,Floor Area (sqm),Flat Model,Lease Commence Date,Remaining Lease,Resale Price,Resale Registration Date
0,324,Ang Mo Kio,10 to 12,98.00,New Generation,1978,54 years,"$553,000.00",Oct 2023
1,122,Ang Mo Kio,07 to 09,92.00,New Generation,1978,54 years,"$492,000.00",Sep 2023
2,210,Ang Mo Kio,01 to 03,91.00,New Generation,1977,52 years,"$470,000.00",Sep 2023
3,348,Ang Mo Kio,07 to 09,89.00,New Generation,1978,53 years,"$500,000.00",Sep 2023
4,134,Ang Mo Kio,07 to 09,98.00,New Generation,1978,54 years,"$550,000.00",Aug 2023
5,234,Ang Mo Kio,01 to 03,102.00,Model A,1993,68 years,"$550,000.00",Aug 2023
6,322,Ang Mo Kio,10 to 12,98.00,New Generation,1977,52 years,"$635,000.00",Aug 2023
7,327,Ang Mo Kio,01 to 03,98.00,New Generation,1977,52 years,"$488,000.00",Aug 2023
8,426,Ang Mo Kio,07 to 09,92.00,New Generation,1978,54 years,"$493,888.00",Aug 2023
9,102,Ang Mo Kio,10 to 12,112.00,Model A,1993,68 years,"$650,000.00",Jul 2023


In [ ]:
df = write_to_dataframe(driver_street, params_street, headers_street)

Ang Mo Kio Ave 3
params["street_val"] is true
54 years
54 years
52 years
53 years
54 years
68 years
52 years
52 years
54 years
68 years
52 years
53 years
53 years
53 years
54 years
54 years
54 years
53 years
54 years
52 years
53 years
53 years
53 years
53 years
55 years
54 years
54 years
54 years
54 years
53 years
53 years
55 years
54 years
53 years
54 years
54 years
55 years
54 years
54 years
54 years
54 years
54 years
54 years
54 years
53 years
54 years
54 years
54 years
55 years
53 years
54 years
55 years
54 years
54 years
53 years
55 years


In [ ]:
df

,Block,HDB Town,Storey,Floor Area (sqm),Flat Model,Lease Commence Date,Remaining Lease,Resale Price,Resale Registration Date
0,324,Ang Mo Kio,10 to 12,98.00,New Generation,1978,54 years,"$553,000.00",Oct 2023
1,122,Ang Mo Kio,07 to 09,92.00,New Generation,1978,54 years,"$492,000.00",Sep 2023
2,210,Ang Mo Kio,01 to 03,91.00,New Generation,1977,52 years,"$470,000.00",Sep 2023
3,348,Ang Mo Kio,07 to 09,89.00,New Generation,1978,53 years,"$500,000.00",Sep 2023
4,134,Ang Mo Kio,07 to 09,98.00,New Generation,1978,54 years,"$550,000.00",Aug 2023
5,234,Ang Mo Kio,01 to 03,102.00,Model A,1993,68 years,"$550,000.00",Aug 2023
6,322,Ang Mo Kio,10 to 12,98.00,New Generation,1977,52 years,"$635,000.00",Aug 2023
7,327,Ang Mo Kio,01 to 03,98.00,New Generation,1977,52 years,"$488,000.00",Aug 2023
8,426,Ang Mo Kio,07 to 09,92.00,New Generation,1978,54 years,"$493,888.00",Aug 2023
9,102,Ang Mo Kio,10 to 12,112.00,Model A,1993,68 years,"$650,000.00",Jul 2023


In [22]:
import smtplib, ssl

port = 465  # For SSL
smtp_server = "smtp.gmail.com"
sender_email = 'hdbresalealertservice@gmail.com'
receiver_email  = 'chngyuanlong@gmail.com'
password = "qcbx wfzf eysm xlxk"
message = """\
Subject: Hi there

This message is sent from Python."""

context = ssl.create_default_context()
with smtplib.SMTP_SSL(smtp_server, port, context=context) as server:
    server.login(sender_email, password)
    server.sendmail(sender_email, receiver_email, message)

In [20]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
 
message = MIMEMultipart()
message["To"] = 'To line here.'
message["From"] = 'From line here.'
message["Subject"] = 'Subject line here.'

title = '<b> Title line here. </b>'
messageText = MIMEText('''Message body goes here.''','html')
message.attach(messageText)

email = 'hdbresalealertservice@gmail.com'
password = '-[=]_{+}'

server = smtplib.SMTP('smtp.gmail.com:587')
server.ehlo('Gmail')
server.starttls()
server.login(email,password)
fromaddr = 'hdbresalealertservice@gmail.com'
toaddrs  = 'chngyuanlong@gmail.com'
server.sendmail(fromaddr,toaddrs,message.as_string())

server.quit()

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials b8-20020a170902b60800b001cc32261bdcsm3524740pls.248 - gsmtp')

In [5]:
driver = start_driver()

In [17]:
# street_combobox = driver.find_element(By.NAME, street_title)
street_combobox = driver.find_element(By.NAME, town_title)

In [18]:
street_values = street_combobox.find_elements(By.TAG_NAME,"option")

In [19]:
len(street_values)

28

In [20]:
for street_name in street_values:
    print("- " + street_name.text)

- Select One
- Ang Mo Kio
- Bedok
- Bishan
- Bukit Batok
- Bukit Merah
- Bukit Panjang
- Bukit Timah
- Central Area
- Choa Chu Kang
- Clementi
- Geylang
- Hougang
- Jurong East
- Jurong West
- Kallang/Whampoa
- Marine Parade
- Pasir Ris
- Punggol
- Queenstown
- Sembawang
- Sengkang
- Serangoon
- Tampines
- Tengah
- Toa Payoh
- Woodlands
- Yishun
